In [1]:
# Import das funções necessárias para os cálculos 

from math import sqrt
from numpy import array, zeros, ix_, delete, dot
from numpy.linalg import solve

![](es24-1.png)
![](es24-2.png)

In [2]:
# Valores para a tabela

E1_length = 0.4
E2_length = sqrt(0.4**2 + 0.4**2)
E3_length = sqrt(0.4**2 + 0.4**2)

print(f"O tamanho do elemento 1 é {E1_length:.3f} m")
print(f"O tamanho do elemento 2 é {E2_length:.3f} m")
print(f"O tamanho do elemento 3 é {E3_length:.3f} m")
print()

E1_cos = 0.4/E1_length
E2_cos = 0.4/E2_length
E3_cos = 0.4/E3_length

print(f"O coseno do ângulo entre o elemento 1 e a vertical é {E1_cos:.3f} rad")
print(f"O coseno do ângulo entre o elemento 2 e a vertical é {E2_cos:.3f} rad")
print(f"O coseno do ângulo entre o elemento 3 e a vertical é {E3_cos:.3f} rad")
print()

E1_sen = sqrt(1 - E1_cos**2)
E2_sen = -sqrt(1 - E2_cos**2)
E3_sen = sqrt(1 - E3_cos**2)

print(f"O seno do ângulo entre o elemento 1 e a vertical é {E1_sen:.3f} rad")
print(f"O seno do ângulo entre o elemento 2 e a vertical é {E2_sen:.3f} rad")
print(f"O seno do ângulo entre o elemento 3 e a vertical é {E3_sen:.3f} rad")

O tamanho do elemento 1 é 0.400 m
O tamanho do elemento 2 é 0.566 m
O tamanho do elemento 3 é 0.566 m

O coseno do ângulo entre o elemento 1 e a vertical é 1.000 rad
O coseno do ângulo entre o elemento 2 e a vertical é 0.707 rad
O coseno do ângulo entre o elemento 3 e a vertical é 0.707 rad

O seno do ângulo entre o elemento 1 e a vertical é 0.000 rad
O seno do ângulo entre o elemento 2 e a vertical é -0.707 rad
O seno do ângulo entre o elemento 3 e a vertical é 0.707 rad


# Informações dos nós

| Número do nó 	| X(m) 	| Y(m) 	|
| :-:        	| :-:   | :-: 	|
| 1            	| 0    	| 0    	|
| 2            	| 0    	| 0,4  	|
| 3            	| 0    	| -0,4 	|
| 4            	| 0,4  	| 0    	|

# Propriedades dos elementos

| Número do elemento 	| Incidência (nós que formam o elemento) 	|   	| Área (m²) 	| Módulo de elasticidade (Pa) 	|   C   	|    S   	| Comprimento (m) 	| Numeração dos graus de liberdade 	|   	|   	|   	|
|:------------------:	|:--------------------------------------:	|:-:	|:---------:	|:---------------------------:	|:-----:	|:------:	|:---------------:	|:--------------------------------:	|:-:	|:-:	|:-:	|
|          1         	|                    1                   	| 4 	|   0,0005  	|       $200 \cdot 10^9$      	|   1   	|    0   	|       0,4       	|                 1                	| 2 	| 7 	| 8 	|
|          2         	|                    2                   	| 4 	|   0,0005  	|       $200 \cdot 10^9$      	| 0,707 	| -0,707 	|      0,566      	|                 3                	| 4 	| 7 	| 8 	|
|          3         	|                    3                   	| 4 	|   0,0005  	|       $200 \cdot 10^9$      	| 0,707 	|  0,707 	|      0,566      	|                 5                	| 6 	| 7 	| 8 	|

In [3]:
# Constantes

E = 200e9  # Pa
A = 0.0005 # m²

F1 = 200   # N
F2 = 150   # N

In [4]:
# a) Matriz de rigidez de cada elemento

# K1    1   2   7   8
#    1 1,1 1,2 1,7 1,8
#    2 2,1 2,2 2,7 2,8
#    7 7,1 7,2 7,7 7,8
#    8 8,1 8,2 8,7 8,8

graus_de_liberdade_em_k1 = array([1, 2, 7, 8])-1

k1 = E*A/E1_length * array(([E1_cos**2, E1_cos*E1_sen, -E1_cos**2, -E1_cos*E1_sen], 
                            [E1_cos*E1_sen, E1_sen**2, -E1_cos*E1_sen, -E1_sen**2], 
                            [-E1_cos**2, -E1_cos*E1_sen, E1_cos**2, E1_cos*E1_sen], 
                            [-E1_cos*E1_sen, -E1_sen**2, E1_cos*E1_sen, E1_sen**2]))

# K2    3   4   7   8
#    3 3,3 3,4 3,7 3,8
#    4 4,3 4,4 4,7 4,8
#    7 7,3 7,4 7,7 7,8
#    8 8,3 8,4 8,7 8,8

graus_de_liberdade_em_k2 = array([3, 4, 7, 8])-1

k2 = E*A/E2_length * array(([E2_cos**2, E2_cos*E2_sen, -E2_cos**2, -E2_cos*E2_sen],
                            [E2_cos*E2_sen, E2_sen**2, -E2_cos*E2_sen, -E2_sen**2],
                            [-E2_cos**2, -E2_cos*E2_sen, E2_cos**2, E2_cos*E2_sen],
                            [-E2_cos*E2_sen, -E2_sen**2, E2_cos*E2_sen, E2_sen**2]))

# K3    5   6   7   8
#    5 5,5 5,6 5,7 5,8
#    6 6,5 6,6 6,7 6,8
#    7 7,5 7,6 7,7 7,8
#    8 8,5 8,6 8,7 8,8

graus_de_liberdade_em_k3 = array([5, 6, 7, 8])-1

k3 = E*A/E3_length * array(([E3_cos**2, E3_cos*E3_sen, -E3_cos**2, -E3_cos*E3_sen],
                            [E3_cos*E3_sen, E3_sen**2, -E3_cos*E3_sen, -E3_sen**2],
                            [-E3_cos**2, -E3_cos*E3_sen, E3_cos**2, E3_cos*E3_sen],
                            [-E3_cos*E3_sen, -E3_sen**2, E3_cos*E3_sen, E3_sen**2]))

print(f"A matriz de rigidez do elemento 1 é: \n{k1}")
print()
print(f"A matriz de rigidez do elemento 2 é: \n{k2}")
print()
print(f"A matriz de rigidez do elemento 3 é: \n{k3}")

A matriz de rigidez do elemento 1 é: 
[[ 2.5e+08  0.0e+00 -2.5e+08 -0.0e+00]
 [ 0.0e+00  0.0e+00 -0.0e+00 -0.0e+00]
 [-2.5e+08 -0.0e+00  2.5e+08  0.0e+00]
 [-0.0e+00 -0.0e+00  0.0e+00  0.0e+00]]

A matriz de rigidez do elemento 2 é: 
[[ 88388347.64831841 -88388347.64831842 -88388347.64831841
   88388347.64831842]
 [-88388347.64831842  88388347.64831844  88388347.64831842
  -88388347.64831844]
 [-88388347.64831841  88388347.64831842  88388347.64831841
  -88388347.64831842]
 [ 88388347.64831842 -88388347.64831844 -88388347.64831842
   88388347.64831844]]

A matriz de rigidez do elemento 3 é: 
[[ 88388347.64831841  88388347.64831842 -88388347.64831841
  -88388347.64831842]
 [ 88388347.64831842  88388347.64831844 -88388347.64831842
  -88388347.64831844]
 [-88388347.64831841 -88388347.64831842  88388347.64831841
   88388347.64831842]
 [-88388347.64831842 -88388347.64831844  88388347.64831842
   88388347.64831844]]


In [5]:
# b) Matriz de rigidez global

# K    1   2   3   4   5   6   7   8
#   1 1,1 1,2 1,3 1,4 1,5 1,6 1,7 1,8
#   2 2,1 2,2 2,3 2,4 2,5 2,6 2,7 2,8
#   3 3,1 3,2 3,3 3,4 3,5 3,6 3,7 3,8
#   4 4,1 4,2 4,3 4,4 4,5 4,6 4,7 4,8
#   5 5,1 5,2 5,3 5,4 5,5 5,6 5,7 5,8
#   6 6,1 6,2 6,3 6,4 6,5 6,6 6,7 6,8
#   7 7,1 7,2 7,3 7,4 7,5 7,6 7,7 7,8
#   8 8,1 8,2 8,3 8,4 8,5 8,6 8,7 8,8

K = zeros((8, 8))

K[ix_(graus_de_liberdade_em_k1, graus_de_liberdade_em_k1)] += k1
K[ix_(graus_de_liberdade_em_k2, graus_de_liberdade_em_k2)] += k2
K[ix_(graus_de_liberdade_em_k3, graus_de_liberdade_em_k3)] += k3

print(f"A matriz de rigidez global é: \n{K}")

A matriz de rigidez global é: 
[[ 2.50000000e+08  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.50000000e+08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  8.83883476e+07 -8.83883476e+07
   0.00000000e+00  0.00000000e+00 -8.83883476e+07  8.83883476e+07]
 [ 0.00000000e+00  0.00000000e+00 -8.83883476e+07  8.83883476e+07
   0.00000000e+00  0.00000000e+00  8.83883476e+07 -8.83883476e+07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   8.83883476e+07  8.83883476e+07 -8.83883476e+07 -8.83883476e+07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   8.83883476e+07  8.83883476e+07 -8.83883476e+07 -8.83883476e+07]
 [-2.50000000e+08  0.00000000e+00 -8.83883476e+07  8.83883476e+07
  -8.83883476e+07 -8.83883476e+07  4.26776695e+08  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  8.8

# c) Vetor de forças

$$ P_G = 

\begin{bmatrix}
R_{1_X} \\
R_{1_Y} \\
R_{2_X} \\
R_{2_Y} \\
R_{3_X} \\
R_{3_Y} \\
F_2     \\
-F_1
\end{bmatrix}
$$

In [6]:
# d) Aplicação das condições de contorno e determinação dos deslocamentos nodais
# (eliminar linha e coluna do grau de liberdade que possui a restrição)

graus_de_liberdade_com_restricoes = array([1, 2, 3, 4, 5, 6])-1
graus_de_liberdade_sem_restricoes = array([7, 8])-1

K_reduzido = delete(K, graus_de_liberdade_com_restricoes, 0)
K_reduzido = delete(K_reduzido, graus_de_liberdade_com_restricoes, 1)

P_reduzido = array([F2, -F1])

print(f"A matriz de rigidez global reduzida é: \n{K_reduzido}")
print()
print(f"O vetor de forças globais reduzido é: \n{P_reduzido}")
print()

U = zeros(8)
U[ix_(graus_de_liberdade_sem_restricoes)] += solve(K_reduzido, P_reduzido)

print(f"O vetor de deslocamentos é: \n{U}")

A matriz de rigidez global reduzida é: 
[[4.26776695e+08 0.00000000e+00]
 [0.00000000e+00 1.76776695e+08]]

O vetor de forças globais reduzido é: 
[ 150 -200]

O vetor de deslocamentos é: 
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  3.51471863e-07 -1.13137085e-06]


In [7]:
# e) Calcular as reações de apoio

P = dot(K, U)

print(f"O vetor de forças globais é: \n{P}")
print()
print(f"R1x = {P[0]:.3f} N")
print(f"R1y = {P[1]:.3f} N")
print(f"R2x = {P[2]:.3f} N")
print(f"R2y = {P[3]:.3f} N")
print(f"R3x = {P[4]:.3f} N")
print(f"R3y = {P[5]:.3f} N")

O vetor de forças globais é: 
[ -87.86796564    0.         -131.06601718  131.06601718   68.93398282
   68.93398282  150.         -200.        ]

R1x = -87.868 N
R1y = 0.000 N
R2x = -131.066 N
R2y = 131.066 N
R3x = 68.934 N
R3y = 68.934 N


In [8]:
# f) Determinar a deformação em cada elemento

# Deformação no elemento 1

epsilon1 = 1/E1_length * dot([-E1_cos, -E1_sen, E1_cos, E1_sen], 
                                U[ix_(graus_de_liberdade_em_k1)])

print(f"A deformação no elemento 1 é de {epsilon1:.3e} m")

# Deformação no elemento 2

epsilon2 = 1/E2_length * dot([-E2_cos, -E2_sen, E2_cos, E2_sen],
                                U[ix_(graus_de_liberdade_em_k2)])

print(f"A deformação no elemento 2 é de {epsilon2:.3e} m")

# Deformação no elemento 3

epsilon3 = 1/E3_length * dot([-E3_cos, -E3_sen, E3_cos, E3_sen],
                                U[ix_(graus_de_liberdade_em_k3)])

print(f"A deformação no elemento 3 é de {epsilon3:.3e} m")

A deformação no elemento 1 é de 8.787e-07 m
A deformação no elemento 2 é de 1.854e-06 m
A deformação no elemento 3 é de -9.749e-07 m


In [9]:
# g) Determinar a tensão em cada elemento

# Tensão no elemento 1

# sigma1 = E/E1_length * np.dot([-E1_cos, -E1_sen, E1_cos, E1_sen],
#                                U[np.ix_(graus_de_liberdade_em_k1-1)])

sigma1 = E * epsilon1

print(f"A tensão no elemento 1 é de {sigma1:.3e} Pa")

# Tensão no elemento 2

# sigma2 = E/E2_length * np.dot([-E2_cos, -E2_sen, E2_cos, E2_sen],
#                                U[np.ix_(graus_de_liberdade_em_k2-1)])

sigma2 = E * epsilon2

print(f"A tensão no elemento 2 é de {sigma2:.3e} Pa")

# Tensão no elemento 3

# sigma3 = E/E3_length * np.dot([-E3_cos, -E3_sen, E3_cos, E3_sen],
#                                  U[np.ix_(graus_de_liberdade_em_k3-1)])

sigma3 = E * epsilon3

print(f"A tensão no elemento 3 é de {sigma3:.3e} Pa")

A tensão no elemento 1 é de 1.757e+05 Pa
A tensão no elemento 2 é de 3.707e+05 Pa
A tensão no elemento 3 é de -1.950e+05 Pa


In [10]:
# h) Qual o elemento que está submetido à maior tensão?

maior_tensao = max(sigma1, sigma2, sigma3)

if maior_tensao == sigma1:
    print(f"O elemento 1 está submetido à maior tensão, que é de {maior_tensao:.3e} Pa")
elif maior_tensao == sigma2:
    print(f"O elemento 2 está submetido à maior tensão, que é de {maior_tensao:.3e} Pa")
else:
    print(f"O elemento 3 está submetido à maior tensão, que é de {maior_tensao:.3e} Pa")

O elemento 2 está submetido à maior tensão, que é de 3.707e+05 Pa
